In [1]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import load_cifar_template1
import time

C:\Users\ebish\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.reset_default_graph()

<h1>Define Hyper-perparmeter</h1>

In [36]:
#learning rate
lr = 0.0001
#number of traning steps
num_steps =15
#number of batch_size
batch_size = 256

#network parameters
hidden_layers=2
nh1 = 512
nh2 = 512
#n_hidden_3=512
num_input = 3072
num_classes = 10

<h1>Define Placeholder</h1>

In [37]:
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')

In [38]:
# #old way to do things
# X = tf.placeholder(tf.float32,[None,num_input],name='X')
# Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')

# #Layers weight & bias
# weights = {
#     'W1': tf.Variable(tf.random_normal([num_input, n_hidden_1]),name='W1'),
#     'W2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]),name='W2'),
#    # 'W3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3]),name='W3'),
#     #'W4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4]),name='W4'),

#     'Wout': tf.Variable(tf.random_normal([n_hidden_2, num_classes]),name='Wout')
# }

# biases = {
#     'b1': tf.Variable(tf.zeros(shape=[n_hidden_1]),name='b1'),
#     'b2': tf.Variable(tf.zeros(shape=[n_hidden_2]),name='b2'),
#     #'b3': tf.Variable(tf.zeros(shape=[n_hidden_3]),name='b3'),
# #'b4': tf.Variable(tf.zeros(shape=[n_hidden_4]),name='b4'),

#     'bout': tf.Variable(tf.zeros(shape=[num_classes]),name='bout')
# }

# #define a neural net model
# def neural_net(x):
#     layer_1_out = tf.nn.relu(tf.add(tf.matmul(x,weights['W1']),biases['b1']))
#     layer_2_out = tf.nn.relu(tf.add(tf.matmul(layer_1_out,weights['W2']),biases['b2']))


#     out = tf.add(tf.matmul(layer_2_out,weights['Wout']),biases['bout'])
#     return out

<h1>Define Neural Network Architecture</h1>

In [39]:
#define a neural net model
def neural_net(x):
    layer_1_out = tf.layers.dense(x,units=nh1,activation='relu')
    layer_2_out = tf.layers.dense(layer_1_out,units=nh2,activation='relu')

    out = tf.layers.dense(layer_2_out,units=num_classes)
    return out

<h1>Define cost andoptimization</h1>

In [40]:
#predicted labels
logits = neural_net(X)

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
#define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()


loss_summary=tf.summary.scalar('loss',loss)
accuracy_summary=tf.summary.scalar('accuracy',accuracy)
#file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [41]:
#learning rate=.0001
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_steps):
        #fetch batch
        
        acc=[]
        st=time.time()
        for j in range(1,6):
            for (a,b) in load_cifar_template1.load_preprocessed_training_batch(j,1000):
                batch_x, batch_y = a, b
                #run optimization
                sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
                
                acc.append(sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y}))
        mid=time.time()
        print("Epoch "+str(i)+", Time:{} ".format(mid-st)+", Training Accuracy= {:.3f}".format(np.average(acc)))

                    
        batch_a,batch_b=load_cifar_template1.load_preprocessed_validation_batch()
        val=sess.run(accuracy, feed_dict={X:batch_a, Y:batch_b})
        print("Validation Accuracy:",val)
    print("Training finished!")
    batch_a,batch_b=load_cifar_template1.load_preprocessed_test_batch()
    test_acc=sess.run(accuracy, feed_dict={X:batch_a, Y:batch_b})
    print("Testing Accuracy:",test_acc)
#file_writer.close()

Epoch 0, Time:24.308100700378418 , Training Accuracy= 0.296
Validation Accuracy: 0.3446
Epoch 1, Time:25.596235275268555 , Training Accuracy= 0.371
Validation Accuracy: 0.3696
Epoch 2, Time:23.390486240386963 , Training Accuracy= 0.397
Validation Accuracy: 0.4038
Epoch 3, Time:23.26980996131897 , Training Accuracy= 0.416
Validation Accuracy: 0.4158
Epoch 4, Time:23.795584440231323 , Training Accuracy= 0.436
Validation Accuracy: 0.42
Epoch 5, Time:22.16875195503235 , Training Accuracy= 0.444
Validation Accuracy: 0.4322
Epoch 6, Time:22.947532892227173 , Training Accuracy= 0.457
Validation Accuracy: 0.4448
Epoch 7, Time:24.492541790008545 , Training Accuracy= 0.463
Validation Accuracy: 0.4492
Epoch 8, Time:22.596689224243164 , Training Accuracy= 0.476
Validation Accuracy: 0.4516
Epoch 9, Time:24.00751304626465 , Training Accuracy= 0.480
Validation Accuracy: 0.4602
Epoch 10, Time:22.603589057922363 , Training Accuracy= 0.487
Validation Accuracy: 0.4728
Epoch 11, Time:21.3372585773468 , Tr

In [29]:
#learning rate=.001
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_steps):
        #fetch batch
        
        acc=[]
        st=time.time()
        for j in range(1,6):
            for (a,b) in load_cifar_template1.load_preprocessed_training_batch(j,1000):
                batch_x, batch_y = a, b
                #run optimization
                sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
                
                acc.append(sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y}))
        mid=time.time()
        print("Epoch "+str(i)+", Time:{} ".format(mid-st)+", Training Accuracy= {:.3f}".format(np.average(acc)))

                    
        batch_a,batch_b=load_cifar_template1.load_preprocessed_validation_batch()
        val=sess.run(accuracy, feed_dict={X:batch_a, Y:batch_b})
        print("Validation Accuracy:",val)
    print("Training finished!")
    batch_a,batch_b=load_cifar_template1.load_preprocessed_test_batch()
    test_acc=sess.run(accuracy, feed_dict={X:batch_a, Y:batch_b})
    print("Testing Accuracy:",test_acc)
#file_writer.close()

Epoch 0, Time:19.955665826797485 , Training Accuracy= 0.246
Validation Accuracy: 0.3036
Epoch 1, Time:18.952347993850708 , Training Accuracy= 0.344
Validation Accuracy: 0.376
Epoch 2, Time:18.977282524108887 , Training Accuracy= 0.385
Validation Accuracy: 0.3874
Epoch 3, Time:19.66244888305664 , Training Accuracy= 0.409
Validation Accuracy: 0.397
Epoch 4, Time:18.834661722183228 , Training Accuracy= 0.432
Validation Accuracy: 0.4166
Epoch 5, Time:18.888516426086426 , Training Accuracy= 0.445
Validation Accuracy: 0.4366
Epoch 6, Time:19.36025834083557 , Training Accuracy= 0.458
Validation Accuracy: 0.4432
Epoch 7, Time:19.090977668762207 , Training Accuracy= 0.469
Validation Accuracy: 0.4532
Epoch 8, Time:19.296428442001343 , Training Accuracy= 0.482
Validation Accuracy: 0.4564
Epoch 9, Time:19.26949977874756 , Training Accuracy= 0.489
Validation Accuracy: 0.475
Training finished!
Testing Accuracy: 0.4782


In [35]:
#learning rate=.0001
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_steps):
        #fetch batch
        
        acc=[]
        st=time.time()
        for j in range(1,6):
            for (a,b) in load_cifar_template1.load_preprocessed_training_batch(j,1000):
                batch_x, batch_y = a, b
                #run optimization
                sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
                
                acc.append(sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y}))
        mid=time.time()
        print("Epoch "+str(i)+", Time:{} ".format(mid-st)+", Training Accuracy= {:.3f}".format(np.average(acc)))

                    
        batch_a,batch_b=load_cifar_template1.load_preprocessed_validation_batch()
        val=sess.run(accuracy, feed_dict={X:batch_a, Y:batch_b})
        print("Validation Accuracy:",val)
    print("Training finished!")
    batch_a,batch_b=load_cifar_template1.load_preprocessed_test_batch()
    test_acc=sess.run(accuracy, feed_dict={X:batch_a, Y:batch_b})
    print("Testing Accuracy:",test_acc)
#file_writer.close()

Epoch 0, Time:19.21863293647766 , Training Accuracy= 0.294
Validation Accuracy: 0.3296
Epoch 1, Time:21.773850440979004 , Training Accuracy= 0.368
Validation Accuracy: 0.379
Epoch 2, Time:19.709324598312378 , Training Accuracy= 0.396
Validation Accuracy: 0.393
Epoch 3, Time:19.40014958381653 , Training Accuracy= 0.417
Validation Accuracy: 0.4122
Epoch 4, Time:21.576334953308105 , Training Accuracy= 0.433
Validation Accuracy: 0.4194
Epoch 5, Time:19.98758101463318 , Training Accuracy= 0.444
Validation Accuracy: 0.4292
Epoch 6, Time:19.296427965164185 , Training Accuracy= 0.453
Validation Accuracy: 0.4272
Epoch 7, Time:20.767496824264526 , Training Accuracy= 0.463
Validation Accuracy: 0.4424
Epoch 8, Time:19.67093515396118 , Training Accuracy= 0.470
Validation Accuracy: 0.449
Epoch 9, Time:19.26251983642578 , Training Accuracy= 0.479
Validation Accuracy: 0.4538
Training finished!
Testing Accuracy: 0.4666


# Insights

I tried with different learning rates, but interestingly it didn't seem to matter if I had a learning rate of .01-.0001. All of these produced a testing accuracy of around 50% at around 10 epochs. I did one test where I did 15 epochs since my computer couldn't seem to handle running more than that. 